In [1]:
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
import pandas as pd
import numpy as np

2024-02-06 18:49:41.895428: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-06 18:49:41.922457: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-06 18:49:41.922483: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-06 18:49:41.922497: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-06 18:49:41.927825: I tensorflow/core/platform/cpu_feature_g

In [2]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    tf.config.set_visible_devices(gpus[0], 'GPU')
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
  except RuntimeError as e:
    # Visible devices must be set before GPUs have been initialized
    print(e)

1 Physical GPUs, 1 Logical GPU


2024-02-06 18:49:43.204689: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-06 18:49:43.209093: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-06 18:49:43.209118: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-06 18:49:43.212856: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-06 18:49:43.212888: I tensorflow/compile

In [3]:
df = pd.read_csv('../../../datos/july_23/2023-07-01.csv', nrows=100000)

In [4]:
df

,block_timestamp,nonce,from_address,to_address,value,gas
0,2023-07-01 00:06:47+00:00,461958,0x292f04a44506c2fd49bac032e1ca148c35a478c8,0xf1a7b0f702eaf433e4697b766306bca41985b75a,881375944790000,55000
1,2023-07-01 00:06:47+00:00,129,0x5c80ae7a02b36c392e34f28859c9dc7a4ecb544f,0xef1c6e67703c7bd7107eed8303fbe6ec2554bf6b,6000000000000000,205047
2,2023-07-01 00:06:47+00:00,38,0xc845914efd144c75d2ffeec2f9609b3bebad681e,0x82e0b8cdd80af5930c4452c684e71c861148ec8a,6000000000000000,202640
3,2023-07-01 00:06:47+00:00,199,0x2bf23d3ab2fe916232ac836b42dc542884df45ae,0x881d40237659c251811cec9c364ef91dc08d300c,300000000000000000,352654
4,2023-07-01 00:33:47+00:00,30,0x78d0365355790d602b571dca709564239954aef6,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,10000000000000000,401234
...,...,...,...,...,...,...
99995,2023-07-01 12:12:47+00:00,1,0xa37602a8f3d0a28a194477f1f97e03a4f76aa0c4,0xb5d85cbf7cb3ee0d56b3bb207d5fc4b82f43f511,8131588964253603,21000
99996,2023-07-01 12:12:47+00:00,1,0x03f9edbbae29a3eed80f0724fef91a8045e73bb0,0xb5d85cbf7cb3ee0d56b3bb207d5fc4b82f43f511,66626488222879315,21000
99997,2023-07-01 09:36:35+00:00,1,0x46945e6f687317841bdbb06dcad812f4f660da40,0xec554417cb64c7433bb2f84a46b9ad633dd457c6,150576440261296251,21000
99998,2023-07-01 09:36:35+00:00,1,0x7096b6674f74bd55a9ccad314b69d84bf2a94c32,0x12ec7e9ebf97f7393de7e6c3c920ec03c4edcf02,6483811281420584,21000


In [5]:
df.columns

Index(['block_timestamp', 'nonce', 'from_address', 'to_address', 'value',
       'gas'],
      dtype='object')

In [6]:
len(set(df['from_address']).union(set(df['to_address'])))

90256

In [7]:
df = df[~df.to_address.isna()]
df = df[~df.from_address.isna()]

In [8]:
ids = {}

for i, id in enumerate(set(df['from_address']).union(set(df['to_address']))):
    ids[id] = i

In [9]:
ids

{'0x7be89c57f70048e22108402f2022f3bde1f5a15f': 0,
 '0x8f5df0872f19b6c8c416294c69afa3c4fe0f9654': 1,
 '0x5ef50c7c2f95892a2d818b68d353788e678dab4b': 2,
 '0x646d5db5138d856f72b6bce0fa7d5ea28a21eebb': 3,
 '0x2c9ddab06919332ed924d6a3b87e49f0c757a01d': 4,
 '0x4c3db4cc16c8d5e95848a5f173661433de0bbb2f': 5,
 '0xd262e92a66b7015ec1f7dae3c854bac4725086ed': 6,
 '0xe42830739d81a30d5b78d506c726622d9ed0cbc7': 7,
 '0x72a018f8dd170e383ae9d66192bebc7ddcb38557': 8,
 '0x29b88ee1b8592e26b3876aa0546dfa6e7754da30': 9,
 '0xb8038a8b06f881caca8a119aa0f0a470e7a8168b': 10,
 '0x17968015c139abd66799463a0f06917ca668bbdf': 11,
 '0x4ea8b7833fc9257f85c617442977924cc5f01cb8': 12,
 '0x98cb430564feed2464b353338eae1895e974c437': 13,
 '0x1ae7664b4c5128a3fb668dcdc246d24c86f95a98': 14,
 '0x0775583cc57c8b9e6c0c76fe7fb56ca3daa41092': 15,
 '0x9fce92d090fabb6187f1e11e10d082081cbf06e1': 16,
 '0xa1e461d803c6cdaefee5fecd5201938f133d6a5e': 17,
 '0xb437f7ab759c0b38806c9e1d4b259b7e4cf8a53a': 18,
 '0xab5701e974dbeca876328fc8fa2d7610a4c42

In [10]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Dense, BatchNormalization, Lambda

# el 1 representa a cualquier indice, un tensor de una dimensión
input_aux = Input(1)

# el 128 es arbitrario, podría ser de cualquier valor
x = Embedding(len(ids), 128)(input_aux)

# TODO: investigar que es esto
x = Dense(64, activation='tanh')(x)

# lleva todos los valores entre -1 y 1
output_aux = BatchNormalization()(x)

model_aux = Model(input_aux, output_aux)
model_aux.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 embedding (Embedding)       (None, 1, 128)            11552128  
                                                                 
 dense (Dense)               (None, 1, 64)             8256      
                                                                 
 batch_normalization (Batch  (None, 1, 64)             256       
 Normalization)                                                  
                                                                 
Total params: 11560640 (44.10 MB)
Trainable params: 11560512 (44.10 MB)
Non-trainable params: 128 (512.00 Byte)
_________________________________________________________________


In [11]:

# son los 3 inputs que representan las 3 cabezas de la red, lo que se conoce como siamesa
input_layer_anchor = Input(1)
input_layer_positive = Input(1)
input_layer_negative = Input(1)

x_a = model_aux(input_layer_anchor)
x_p = model_aux(input_layer_positive)
x_n = model_aux(input_layer_negative)

# un tensor es ... investigar bien. Puede ser cualquier cosa, literalmente.
merged_output = Lambda(lambda tensors: tf.stack(tensors, axis=-1))([x_a, x_p, x_n])

model = Model([input_layer_anchor, input_layer_positive, input_layer_negative], merged_output)

In [12]:
from keras.utils import Sequence
import numpy as np

class GeneratorTriplet(Sequence):
    def __init__(self, df, ids, batch_size):
        self.df = df
        self.act_index = 0
        self.ids = ids
        self.batch_size = batch_size
        self.limit = int(np.ceil(len(self.df) / self.batch_size))

    def __len__(self):
        return int(np.ceil(len(self.df) / self.batch_size))
    
    
    def __iter__(self):
        return self
    
    def __next__(self):
        if self.act_index < self.limit:
            resultado = self.__getitem__(self.act_index)
            self.act_index += 1
            return resultado
        else:
            raise StopIteration

    def __getitem__(self, index):
        # el Anchor es un A, el positive es un B y el negative es un C. Queremos acercar A y B tanto como sea posible alejando a C. Podría suceder que sea un vecino, pero la probabilidad es baja porque es al azar.
        # Si llegara a elegir algún vecino mal, de casualidad, en una siguiente epoch debería corregirse o inclusive en una misma epoch en una siguiente iteración del generador.
        init = index * self.batch_size
        end = (index + 1) * self.batch_size

        batch = self.df[init:end]
        negative = self.df.sample(len(batch))

        anchor = np.array(batch['from_address'].apply(lambda x: self.ids.get(x)))
        positive = np.array(batch['to_address'].apply(lambda x: self.ids.get(x)))
        negative = np.array(negative['to_address'].apply(lambda x: self.ids.get(x)))
        
        anchor = tf.convert_to_tensor(anchor)
        positive = tf.convert_to_tensor(positive)
        negative = tf.convert_to_tensor(negative)
        
        # el fake target simunla lo que sería aprendizaje supervisado        
        fake_target = tf.convert_to_tensor(np.array([1]*self.batch_size))

        return ([anchor, positive, negative], [fake_target]*3)

In [13]:
generator = GeneratorTriplet(df, ids, 64)

In [14]:
class TripletCustom(tf.keras.losses.Loss):
    def __init__(self, margin=0.001, **kwargs):
        super().__init__(**kwargs)
        self.margin = margin

    @tf.function
    def __call__(self, y, y_pred, sample_weight=None):
        anchor, positive, negative =  tf.split(y_pred, num_or_size_splits=3, axis=-1)

        if tf.math.is_nan(anchor) is True or tf.math.is_nan(positive) is True or tf.math.is_nan(negative) is True:
            raise Exception("Falla por aca")
        
        distance_positive = tf.norm(anchor - positive, axis=-1)
        distance_negative = tf.norm(anchor - negative, axis=-1)

        if tf.math.is_nan(distance_positive) is True or tf.math.is_nan(distance_negative) is True:
            raise Exception("Falla por aca 2")

        loss = tf.maximum(distance_positive - distance_negative + self.margin, 0.0)

        if tf.math.is_nan(loss) is True:
            raise Exception("Falla por aca 3")

        resultado = tf.reduce_mean(loss)

        if tf.math.is_nan(resultado) is True:
            raise Exception("Falla por aca 4")

        return resultado

loss = TripletCustom()    

model.compile(
    optimizer=Adam(0.00001),
    loss=loss
)

In [15]:
model.fit(generator, epochs=10)

Epoch 1/10


2024-02-06 18:49:47.410525: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7fbf9075f560 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-02-06 18:49:47.410566: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3070 Ti, Compute Capability 8.6
2024-02-06 18:49:47.415093: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-02-06 18:49:47.551068: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8700
2024-02-06 18:49:47.607867: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1563/1563 [==============================] - 25s 14ms/step - loss: 0.2680
Epoch 2/10
1563/1563 [==============================] - 18s 12ms/step - loss: 0.2430
Epoch 3/10
1563/1563 [==============================] - 18s 11ms/step - loss: 0.2215
Epoch 4/10
1563/1563 [==============================] - 18s 11ms/step - loss: 0.2027
Epoch 5/10
 300/1563 [====>.........................] - ETA: 14s - loss: 0.1977

In [ ]:
next(generator)

([<tf.Tensor: shape=(64,), dtype=int64, numpy=
  array([62001, 89568, 22720, 72776, 41043, 44950, 72931, 61409, 28717,
         35608, 31400, 83683, 55442, 62804, 32838, 55022, 36433, 58270,
         67205,  1468, 58845, 66836, 59859, 27196, 83683, 83683, 21738,
         55227,  7906, 78743,  9417, 15096, 46910, 69292, 47442, 27642,
         88302, 17424,  5905, 21715, 72743, 35345, 12379, 44501, 47126,
         33839, 82294, 60403, 10820, 17125, 31982, 35963, 39785, 42369,
         88204, 34337, 13624, 11305,  6826, 48519, 20104, 12169, 63226,
         21809])>,
  <tf.Tensor: shape=(64,), dtype=int64, numpy=
  array([29322, 78156, 10750, 78277, 40892, 88226, 88226, 40892, 67618,
         40892, 66458, 86618, 37955, 47379, 31892,   397, 49175,  9669,
         28870, 88226, 40892, 56392, 40325, 29915, 86618, 86618, 29915,
         29915,  8002, 61399, 45686, 40892, 88226, 88222, 88226, 86302,
         10541, 40892, 88226, 48957, 88226, 40892, 88226, 88226,  6796,
         67864, 37955, 

In [ ]:
# input_layer = Input(shape=(2,))
# dense_layer = Dense(units=1, activation='linear')(input_layer)
# model = Model(inputs=input_layer, outputs=dense_layer)
# model.compile(optimizer='adam', loss='mean_squared_error')
# model.summary()
# y_train = np.random.rand(100)
# print(model.fit(combined_tensor, y_train, epochs=1, batch_size=32))